In [16]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


In [2]:
# %pip install langchain_community faiss-cpu
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

In [9]:

# 1. Load Data
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

# 2️. Text Split
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(documents)
#print(split_docs)

# 3️. Indexing (벡터 저장)
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
# 로컬 파일로 저장
vectorstore.save_local("faiss_index")

# 4️. Retrieval (유사 문서 검색)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
# **질문(쿼리)**에 대해 유사한 문서를 검색하는 역할
retrieved_docs = retriever.invoke("소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
#print(retrieved_docs)

# 5️. Generation (LLM 응답 생성)
llm = ChatOpenAI(model="gpt-4o")
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
#print(context)

In [10]:

response_context = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}")
print('context 적용한 결과')
pprint(response_context.content)


context 적용한 결과
('소득세법 제12조에 따르면 여러 종류의 소득이 비과세소득으로 분류되어 소득세가 부과되지 않습니다. 주요 비과세소득 항목은 다음과 '
 '같습니다:\n'
 '\n'
 '1. **공익신탁의 이익**: 「공익신탁법」에 따라 발생하는 공익신탁의 이익은 비과세됩니다.\n'
 '\n'
 '2. **일부 사업소득**:\n'
 '   - **농업 소득**: 논과 밭을 작물 생산에 이용하여 발생하는 소득은 비과세됩니다.\n'
 '   - **주택임대소득**: 1개의 주택을 소유한 자의 주택임대소득 중 일부는 비과세됩니다. 단, 제99조에 따른 기준시가가 12억원을 '
 '초과하는 주택 및 국외에 소재하는 주택의 임대소득은 과세 대상입니다. 또한, 해당 과세기간에 총수입금액의 합계액이 2천만원 이하인 자의 '
 '주택임대소득(2018년 12월 31일 이전까지 발생하는 소득)은 비과세됩니다.\n'
 '   - **농어가부업소득**: 대통령령으로 정하는 농어가부업소득이 비과세됩니다.\n'
 '   - **전통주의 제조 소득**: 대통령령으로 정하는 전통주의 제조에서 발생하는 소득은 비과세됩니다.\n'
 '   - **임목의 벌채 또는 양도 소득**: 조림기간 5년 이상인 임지의 임목 벌채 또는 양도로 발생하는 연 600만원 이하의 소득은 '
 '비과세됩니다. 조림기간 및 세액 계산에 관한 사항은 대통령령에 따릅니다.\n'
 '   - **작물재배업 소득**: 대통령령으로 정하는 작물재배업에서 발생하는 소득은 비과세됩니다.\n'
 '\n'
 '이 항목들은 여러 법률 개정을 거쳐 현재의 규정에 이르게 되었으며, 구체적인 사항은 관련 대통령령에 따릅니다. 이를 통해 특정 조건을 '
 '만족하면 일부 소득은 비과세 혜택을 받을 수 있습니다.')


In [11]:

response = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
print('context 적용하지 않은 결과')
pprint(response.content)

context 적용하지 않은 결과
('소득세법에서 비과세소득은 과세 대상에서 제외되어 세금을 부과하지 않는 소득을 의미합니다. 한국 소득세법에 따라 비과세소득으로 인정되는 '
 '항목에는 다음과 같은 것들이 포함될 수 있습니다:\n'
 '\n'
 '1. **국민연금 및 기타 공적연금의 일부**: 일정 금액 이하의 연금소득은 비과세로 처리될 수 있습니다.\n'
 '   \n'
 '2. **고용보험 및 산업재해보상보험에 의한 급여**: 예를 들어 실업급여나 산업재해로 인한 보상금 등이 해당됩니다.\n'
 '\n'
 '3. **장애인 연금**: 장애인 복지를 위한 일부 연금은 비과세될 수 있습니다.\n'
 '\n'
 '4. **학자금 및 장학금**: 교육 지원을 목적으로 하는 장학금 중 일정 조건을 충족하는 경우 비과세로 인정됩니다.\n'
 '\n'
 '5. **일정한 저축성 보험의 환급금**: 일정 요건을 충족하는 저축성 보험의 환급금이 비과세될 수 있습니다.\n'
 '\n'
 '6. **국가 및 지방자치단체가 지급하는 보조금 중 일부**: 특정한 요건 하에서 제공되는 보조금은 비과세 소득으로 인정됩니다.\n'
 '\n'
 '7. **기타 법령에 의해 비과세로 규정된 소득**: 법령에 따라 추가적으로 정해진 다양한 비과세 소득 항목들이 있을 수 있습니다.\n'
 '\n'
 '자세한 사항은 법령 개정에 따라 변경될 수 있으므로, 최신 정보는 한국 국세청이나 관련 법령 문서를 통해 확인하는 것이 좋습니다.')


### 1차 개선

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. 데이터 로드 (기존과 동일)
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

# 2. 텍스트 분할 개선
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 크기 증가
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""],  # 자연스러운 분할을 위한 구분자
    length_function=len,
    is_separator_regex=False,
)
split_docs = splitter.split_documents(documents)

# 3. 인덱싱 (벡터 저장)
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
vectorstore.save_local("./db/faiss_index")

# 4. 검색 개선
retriever = vectorstore.as_retriever(
    search_type="mmr",  # 최대 다양성 검색
    search_kwargs={"k": 5, "fetch_k": 10}  # 더 많은 결과 검색
)

# 5. 프롬프트 엔지니어링
def generate_prompt(query, context):
    return f"""다음은 소득세법 비과세소득 관련 조항입니다. 문맥을 고려하여 질문에 답변하세요.

[관련 조항]
{context}

[질문]
{query}

[답변 요구사항]
- 비과세소득 유형을 계층적으로 구분하여 설명
- 각 항목별 구체적인 조건 명시
- 법조문의 항, 호, 목 번호를 포함
- 500자 이내로 간결하게 요약"""

# 검색 및 응답 생성
query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

llm = ChatOpenAI(model="gpt-4o", temperature=0.3)  # 창의성 낮춤
response = llm.invoke(generate_prompt(query, context))

print('개선된 결과:')
pprint(response.content)

개선된 결과:
('소득세법 제12조에 따른 비과세소득은 다음과 같이 구분됩니다:\n'
 '\n'
 '1. **공익신탁 이익**: 공익신탁법에 따른 이익(제12조 1호).\n'
 '\n'
 '2. **사업소득**: \n'
 '   - 작물 생산을 위한 논밭 임대 소득(제12조 2호 가목).\n'
 '   - 1주택 소유자의 일정 조건 주택임대소득(제12조 2호 나목).\n'
 '   - 농어가부업소득, 전통주 제조 소득 등 대통령령으로 정하는 소득(제12조 2호 다~사목).\n'
 '\n'
 '3. **근로소득 및 퇴직소득**:\n'
 '   - 국외근로 급여, 특정 보험료, 연장근로 수당 등(제12조 3호 거~어목).\n'
 '\n'
 '4. **기타소득**:\n'
 '   - 보훈급여금, 상금, 직무발명보상금 등(제12조 5호 가~자목).\n'
 '\n'
 '각 항목은 대통령령 등에서 정한 구체적인 조건을 충족해야 비과세 혜택을 받을 수 있습니다.')


### 2차 개선

In [19]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. Load Data
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

print("=== 원본 문서 길이 ===")
print(f"전체 문서 길이: {len(documents[0].page_content)} 글자")

# 2. Text Split 개선
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # 500 → 800 (법령 조항이 길어서)
    chunk_overlap=150,  # 50 → 150 (맥락 보존 강화)
    separators=["\n\n", "\n", ". ", " ", ""]  # 법령 구조에 맞는 분리자
)
split_docs = splitter.split_documents(documents)

print(f"분할된 문서 수: {len(split_docs)}개")
print("=== 분할 예시 ===")
for i, doc in enumerate(split_docs[:3]):
    print(f"Chunk {i+1} ({len(doc.page_content)}글자): {doc.page_content[:100]}...")

# 3. Indexing
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
vectorstore.save_local("db/faiss_index")

# 4. Retrieval 개선
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 6}  # 2 → 6으로 증가
)

query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)

print(f"\n=== 검색된 문서 ({len(retrieved_docs)}개) ===")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}: {doc.page_content[:200]}...")
    print("---")

# 5. Generation - 개선된 프롬프트
llm = ChatOpenAI(model="gpt-4o", temperature=0)
context = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])

# 개선된 프롬프트 - 더 구체적인 지시사항
improved_prompt = f"""
당신은 세무 전문가입니다. 아래 소득세법 제12조 조항을 바탕으로 질문에 답변해주세요.

질문: {query}

법령 조항:
{context}

다음 형식으로 답변해주세요:
1. 비과세소득의 정의
2. 주요 비과세소득 항목들을 다음과 같이 분류:
   - 사업소득 관련
   - 근로소득/퇴직소득 관련  
   - 연금소득 관련
   - 기타소득 관련
3. 각 항목별 구체적인 조건이나 한도액 명시

답변은 법조문을 인용하면서 구체적으로 작성해주세요.
"""

# 비교용 - 기존 방식
simple_prompt = f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}"

print("\n=== 개선된 프롬프트로 답변 ===")
response_improved = llm.invoke(improved_prompt)
pprint(response_improved.content)

print("\n" + "="*50)
print("=== 기존 프롬프트로 답변 ===")
response_simple = llm.invoke(simple_prompt)
pprint(response_simple.content)

# 추가 개선: 다른 검색 방식 시도
print("\n" + "="*50)
print("=== 검색 방식 개선 테스트 ===")


=== 원본 문서 길이 ===
전체 문서 길이: 4969 글자
분할된 문서 수: 8개
=== 분할 예시 ===
Chunk 1 (738글자): 제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2....
Chunk 2 (636글자): 다. 대통령령으로 정하는 농어가부업소득
    라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득
    마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양...
Chunk 3 (792글자): 라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금, 상병보상금(傷病補償金), 일시보상금, 장해보상금, 유족보상금, 행방불명보상금, ...

=== 검색된 문서 (6개) ===
문서 1: 제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 2...
---
문서 2: 2) 대학의 교직원 또는 대학과 고용관계가 있는 학생이 소속 대학에 설치된 「산업교육진흥 및 산학연협력촉진에 관한 법률」 제25조에 따른 산학협력단(이하 이 조에서 “산학협력단”이라 한다)으로부터 같은 법 제32조제1항제4호에 따라 받는 보상금
    저. 대통령령으로 정하는 복리후생적 성질의 급여
4. 연금소득 중 다음 각 목의 어느 하나에 해당하는 소득...
---
문서 3: 나. 「국가보안법」에 따라 받는 상금과 보로금
    다. 「상훈법」에 따른 훈장과 관련하여 받는 부상(副賞)이나 그 밖에 대통령령으로 정하는 상금과 부상
    라. 종업원등 또는 대학의 교직원이 퇴직한 후에 사용

In [15]:

# MMR(Maximum Marginal Relevance) 검색 - 다양성 확보
retriever_mmr = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 6, "fetch_k": 20}
)
retrieved_docs_mmr = retriever_mmr.invoke(query)
context_mmr = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs_mmr)])

response_mmr = llm.invoke(f"""
{query}

법령 조항 (MMR 검색):
{context_mmr}

위 법령을 바탕으로 비과세소득 항목들을 체계적으로 정리해주세요.
""")

print("=== MMR 검색 결과 ===")
pprint(response_mmr.content)

=== MMR 검색 결과 ===
('소득세법 제12조에 따른 비과세소득 항목은 다음과 같이 체계적으로 정리할 수 있습니다.\n'
 '\n'
 '1. **공익신탁의 이익**\n'
 '   - 「공익신탁법」에 따른 공익신탁의 이익\n'
 '\n'
 '2. **사업소득**\n'
 '   - 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n'
 '   - 1개의 주택을 소유하는 자의 주택임대소득 (단, 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 '
 '제외)\n'
 '   - 대통령령으로 정하는 농어가부업소득\n'
 '   - 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n'
 '   - 조림기간 5년 이상인 임지의 임목의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액\n'
 '   - 대통령령으로 정하는 작물재배업에서 발생하는 소득\n'
 '   - 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n'
 '\n'
 '3. **근로소득과 퇴직소득**\n'
 '   - 대통령령으로 정하는 복무 중인 병이 받는 급여\n'
 '   - 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n'
 '   - 「산업재해보상보험법」에 따라 수급권자가 받는 각종 보상금\n'
 '   - 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 각종 보상금\n'
 '   - 「고용보험법」에 따라 받는 실업급여 등\n'
 '   - 「국민연금법」에 따라 받는 반환일시금 및 사망일시금\n'
 '   - 「공무원연금법」 등 관련 법령에 따라 받는 각종 보상금 및 급여\n'
 '   - 대통령령으로 정하는 학자금\n'
 '\n'
 '4. **연금소득**\n'
 '   - 「국민연금법」, 「공무원연금법」 등 공적연금 관련법에 따라 받는 각종 연금\n'
 '   - 「산업재해보상보험법」에 따라 받는 각종 연금\n'
 '   - 「국군포로의 송환 및 대우 등에 관한 법률」에 따른 연금\n'
 '\n'
 '

### RAG_PDF_LOADER

In [20]:
# pip install pypdf

import os
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# PDF 파일 경로 설정
pdf_filepath = 'data/tutorial-korean.pdf'

# 파일 존재 여부 확인 (파일이 없으면 오류 발생)
if not os.path.exists(pdf_filepath):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {pdf_filepath}")


In [22]:

try:
    # 1. PDF 파일 로드
    loader = PyPDFLoader(pdf_filepath)  # PDF 파일을 로드할 객체 생성
    docs = loader.load()  # 문서를 전체 로드

    # 총 문서 개수 출력
    print(f"총 {len(docs)}개의 문서가 로드 되었습니다.")

    #  첫 번째 문서의 메타데이터 출력
    print("첫 번째 문서 메타데이터:")
    print(json.dumps(docs[0].metadata, indent=2, ensure_ascii=False))

    # 특정 인덱스(10번째) 문서의 내용 확인 (존재할 경우)
    if len(docs) > 10:
        print("\n10번째 문서 내용:", type(docs[10]))
        print(docs[10])  # 10번째 문서 출력

    #  2. 텍스트 분할 (200자 단위, 중첩 없음)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
    split_docs = loader.load_and_split(text_splitter=text_splitter)  # 분할된 문서 로드

    # 분할된 문서 개수 출력
    print(f"\n분할된 문서의 개수: {len(split_docs)} 타입: {type(split_docs)}")

    # 10번째 분할된 문서 내용 출력 (존재할 경우)
    if len(split_docs) > 10:
        print("\n10번째 분할된 문서:")
        print(split_docs[10])

    # 3. Lazy Load 방식으로 문서 로드
    print("\nLazy Load 방식으로 문서 로드:")
    for i, doc in enumerate(loader.lazy_load()):
        if i < 5:  # 너무 많은 출력 방지 (예제: 처음 5개만 출력)
            print(json.dumps(doc.metadata, indent=2, ensure_ascii=False))

except Exception as e:
    # 오류 발생 시 메시지 출력
    print(f"오류 발생: {e}")

총 39개의 문서가 로드 되었습니다.
첫 번째 문서 메타데이터:
{
  "producer": "Acrobat Distiller with ezUniHFT",
  "creator": "PScript5.dll Version 5.2",
  "creationdate": "2005-04-26T15:21:34+09:00",
  "moddate": "2005-04-26T15:21:34+09:00",
  "author": "Owner",
  "title": "<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>",
  "source": "data/tutorial-korean.pdf",
  "total_pages": 39,
  "page": 0,
  "page_label": "1"
}

10번째 문서 내용: <class 'langchain_core.documents.base.Document'>
page_content='11
그림 5 와 같이 getRoom과 setRoom 메소드들은 각각 staff 멤버의 방번호(room 
number)를 설정하고 반환하는 동작을 합니다. getRoom 메소드를 호출해 봅시다. 객체 메뉴의 
getRoom 메소드를 선택하여 실행합니다. 그러면 대화상자에서 실행 결과를 볼 수 있을 것
입니다(그림 6). 그림 6과 같이 결과의 내용이 "(unknown room)"이 됩니다. 왜냐하면, Staff 
객체에 대한 방번호를 지정하지 않았기 때문입니다.
                     
그림 6 : 메소드 호출 결과
슈퍼 클래스에서 상속된 메소드들은 서브 메뉴(inherited from Person)에서 선택하여 사용
할 수 있습니다. 객체 팝업메뉴의 상단에는 두개의 서브메뉴
3)가 있을 것입니다. 하나는 
Object 클래스로부터 상속 받은 메소드들이고 다른 하나는 Person 클래스로부터 상속 받은 
메소드들입니다(그림 5). 따라서, 서브 메뉴를 선택하면 getName과 같은 Person 

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

print("==> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader('./data/tutorial-korean.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

print("==> 2. 문서 분할 → 작은 청크로 나누기")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # 청크 크기 (한국어 최적화)
    chunk_overlap=200,      # 중복 부분 (맥락 보존)
    separators=["\n\n", "\n", ".", " ", ""] # 자연스러운 분할을 위한 구분자
)

chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")
print(f"  평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")

print("==> 3. 벡터화 → 임베딩으로 변환")
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  # 고성능 임베딩 모델
    dimensions=1536
)

print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)
print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")

print("===> 5. 검색 → 질문과 유사한 문서 찾기")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}  # 상위 6개 관련 문서 검색
)
print(" Retriever 설정 완료")

print("===> 6. 생성 → LLM으로 답변 생성")
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.1,
    max_tokens=1500
)

# 한국어 최적화 프롬프트
prompt_template = """
당신은 BlueJ 프로그래밍 환경 전문가입니다. 
아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.

문서 내용:
{context}

질문: {question}

답변 규칙:
1. 문서 내용만을 근거로 답변하세요
2. 단계별 설명이 필요하면 순서대로 작성하세요  
3. 구체적인 메뉴명, 버튼명을 포함하세요
4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요

답변:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
print(" 프롬프트 설정 완료")

# ===================================
# 7. QA 체인 생성
# ===================================
print("\n ===> 7.  QA 체인 생성...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
print("  RAG 파이프라인 구축 완료!")

# ===================================
# 8. 테스트 질문들
# ===================================
test_questions = [
    "BlueJ에서 객체를 생성하는 방법은 무엇인가요?",
    "컴파일 오류가 발생했을 때 어떻게 확인할 수 있나요?", 
    "디버깅을 위해 중단점을 설정하는 방법을 알려주세요",
    "코드패드는 무엇이고 어떻게 사용하나요?",
    "애플릿을 만들고 실행하는 방법을 설명해주세요"
]

print("\n" + "=" * 60)
print(" RAG 시스템 테스트")
print("=" * 60)

# ===================================
# 9. 질문 및 답변 실행
# ===================================
for i, question in enumerate(test_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)


==> 1. 문서 로딩 → PDF 읽기...
  총 39페이지 로드 완료
==> 2. 문서 분할 → 작은 청크로 나누기
  66개 청크 생성 완료
  평균 청크 길이: 695자
==> 3. 벡터화 → 임베딩으로 변환
==> 4. 저장 → FAISS 벡터스토어에 저장
 FAISS 벡터스토어 생성 완료 (66개 벡터)
===> 5. 검색 → 질문과 유사한 문서 찾기
 Retriever 설정 완료
===> 6. 생성 → LLM으로 답변 생성
 프롬프트 설정 완료

 ===> 7.  QA 체인 생성...
  RAG 파이프라인 구축 완료!

 RAG 시스템 테스트

【테스트 1/5】
 질문: BlueJ에서 객체를 생성하는 방법은 무엇인가요?
 답변 생성 중...

 답변:
--------------------------------------------------
BlueJ에서 객체를 생성하는 방법은 다음과 같습니다:

1. **프로젝트 열기**: 먼저 BlueJ를 시작한 후, 상단 메뉴에서 `Project - Open...`을 선택하여 프로젝트를 엽니다. 예제 프로젝트는 BlueJ 홈 디렉토리의 `examples` 디렉토리에서 찾을 수 있습니다. 예를 들어, `people` 프로젝트를 선택하여 엽니다.

2. **객체 생성**: 
   - 메인 윈도우 중앙에 있는 클래스 아이콘(예: Database, Person, Staff, Student) 중 하나를 선택합니다.
   - 클래스 아이콘에서 마우스 오른쪽 버튼을 클릭(매킨토시의 경우, `ctrl+click`)하여 팝업 메뉴를 엽니다.
   - 팝업 메뉴에서 생성자 함수를 선택합니다. 이때, 생성할 객체의 이름을 입력받는 대화상자가 나타납니다. 기본적으로 제공되는 이름(예: `staff_1`)을 사용할 수 있습니다.
   - `OK` 버튼을 클릭하면 객체가 생성되고, 생성된 객체는 오브젝트 벤치(object bench)에 표시됩니다.

3. **추상 클래스 주의**: 추상 클래스(예: Person 클래스)는 객체를 생성

### RAG - CharacterTextSplitter
- 공백이나 띄어쓰기 기준으로 split

In [4]:
from langchain.text_splitter import CharacterTextSplitter

# ===================================
# 예제 텍스트
# ===================================
text = """RAG는 검색 기반의 텍스트 생성 모델입니다. 기존 언어 모델의 단점을 보완하고, 최신 정보를 제공합니다.
특히, 최신 데이터를 반영하는 데 강력한 기능을 제공합니다. 
RAG는 검색과 생성 단계를 포함합니다. 먼저 관련 문서를 검색하고, 그 다음 검색된 문서를 바탕으로 답변을 생성합니다.
이 방식은 환상(hallucination) 문제를 크게 줄여줍니다. 또한 실시간으로 최신 정보를 활용할 수 있어 매우 유용합니다."""

print(" 원본 텍스트:")
print("-" * 50)
print(text)
print(f"\n 원본 길이: {len(text)}자")

# ===================================
#  다양한 분할 방식 비교
# ===================================

print("\n" + "="*60)
print(" 다양한 CharacterTextSplitter 설정 비교")
print("="*60)

# 기본 설정 (마침표 기준)
print("\n 마침표(.) 기준 분할:")
print("-" * 30)
splitter1 = CharacterTextSplitter(
    chunk_size=50,      # 청크 최대 크기
    chunk_overlap=10,   # 청크 간 중복
    separator="."       # 분할 기준
)
chunks1 = splitter1.split_text(text)

for i, chunk in enumerate(chunks1, 1):
    print(f"청크 {i}: '{chunk.strip()}' (길이: {len(chunk)}자)")


 원본 텍스트:
--------------------------------------------------
RAG는 검색 기반의 텍스트 생성 모델입니다. 기존 언어 모델의 단점을 보완하고, 최신 정보를 제공합니다.
특히, 최신 데이터를 반영하는 데 강력한 기능을 제공합니다. 
RAG는 검색과 생성 단계를 포함합니다. 먼저 관련 문서를 검색하고, 그 다음 검색된 문서를 바탕으로 답변을 생성합니다.
이 방식은 환상(hallucination) 문제를 크게 줄여줍니다. 또한 실시간으로 최신 정보를 활용할 수 있어 매우 유용합니다.

 원본 길이: 235자

 다양한 CharacterTextSplitter 설정 비교

 마침표(.) 기준 분할:
------------------------------
청크 1: 'RAG는 검색 기반의 텍스트 생성 모델입니다' (길이: 24자)
청크 2: '기존 언어 모델의 단점을 보완하고, 최신 정보를 제공합니다' (길이: 32자)
청크 3: '특히, 최신 데이터를 반영하는 데 강력한 기능을 제공합니다' (길이: 32자)
청크 4: 'RAG는 검색과 생성 단계를 포함합니다' (길이: 21자)
청크 5: '먼저 관련 문서를 검색하고, 그 다음 검색된 문서를 바탕으로 답변을 생성합니다' (길이: 43자)
청크 6: '이 방식은 환상(hallucination) 문제를 크게 줄여줍니다' (길이: 36자)
청크 7: '또한 실시간으로 최신 정보를 활용할 수 있어 매우 유용합니다' (길이: 33자)


In [5]:

#  문장 기준 (좀 더 큰 청크)
print("\n 문장 기준 분할 (큰 청크):")
print("-" * 30)
splitter2 = CharacterTextSplitter(
    chunk_size=100,     # 더 큰 청크
    chunk_overlap=50,   # 더 많은 중복
    separator="."
)
chunks2 = splitter2.split_text(text)

for i, chunk in enumerate(chunks2, 1):
    print(f"청크 {i}: '{chunk.strip()}' (길이: {len(chunk)}자)")



 문장 기준 분할 (큰 청크):
------------------------------
청크 1: 'RAG는 검색 기반의 텍스트 생성 모델입니다. 기존 언어 모델의 단점을 보완하고, 최신 정보를 제공합니다.
특히, 최신 데이터를 반영하는 데 강력한 기능을 제공합니다' (길이: 92자)
청크 2: '특히, 최신 데이터를 반영하는 데 강력한 기능을 제공합니다. 
RAG는 검색과 생성 단계를 포함합니다' (길이: 56자)
청크 3: 'RAG는 검색과 생성 단계를 포함합니다. 먼저 관련 문서를 검색하고, 그 다음 검색된 문서를 바탕으로 답변을 생성합니다' (길이: 66자)
청크 4: '먼저 관련 문서를 검색하고, 그 다음 검색된 문서를 바탕으로 답변을 생성합니다.
이 방식은 환상(hallucination) 문제를 크게 줄여줍니다' (길이: 81자)
청크 5: '이 방식은 환상(hallucination) 문제를 크게 줄여줍니다. 또한 실시간으로 최신 정보를 활용할 수 있어 매우 유용합니다' (길이: 71자)


In [6]:

#  줄바꿈 기준
print("\n 줄바꿈(\\n) 기준 분할:")
print("-" * 30)
splitter3 = CharacterTextSplitter(
    chunk_size=80,
    chunk_overlap=0,    # 중복 없음
    separator="\n"
)
chunks3 = splitter3.split_text(text)

for i, chunk in enumerate(chunks3, 1):
    print(f"청크 {i}: '{chunk.strip()}' (길이: {len(chunk)}자)")



 줄바꿈(\n) 기준 분할:
------------------------------
청크 1: 'RAG는 검색 기반의 텍스트 생성 모델입니다. 기존 언어 모델의 단점을 보완하고, 최신 정보를 제공합니다.' (길이: 59자)
청크 2: '특히, 최신 데이터를 반영하는 데 강력한 기능을 제공합니다.' (길이: 33자)
청크 3: 'RAG는 검색과 생성 단계를 포함합니다. 먼저 관련 문서를 검색하고, 그 다음 검색된 문서를 바탕으로 답변을 생성합니다.' (길이: 67자)
청크 4: '이 방식은 환상(hallucination) 문제를 크게 줄여줍니다. 또한 실시간으로 최신 정보를 활용할 수 있어 매우 유용합니다.' (길이: 72자)


In [8]:

#  공백 기준 (단어 단위)
print("\n 공백(' ') 기준 분할 (단어 단위):")
print("-" * 30)
splitter4 = CharacterTextSplitter(
    chunk_size=30,      # 작은 청크
    chunk_overlap=5,
    separator=" "       # 공백으로 분할
)
chunks4 = splitter4.split_text(text)

for i, chunk in enumerate(chunks4[:5], 1):  # 처음 5개만 출력
    print(f"청크 {i}: '{chunk.strip()}' (길이: {len(chunk)}자)")
print(f"... 총 {len(chunks4)}개 청크 생성됨")



 공백(' ') 기준 분할 (단어 단위):
------------------------------
청크 1: 'RAG는 검색 기반의 텍스트 생성 모델입니다. 기존' (길이: 28자)
청크 2: '기존 언어 모델의 단점을 보완하고, 최신 정보를' (길이: 26자)
청크 3: '정보를 제공합니다.
특히, 최신 데이터를 반영하는 데' (길이: 29자)
청크 4: '데 강력한 기능을 제공합니다. 
RAG는 검색과 생성' (길이: 29자)
청크 5: '생성 단계를 포함합니다. 먼저 관련 문서를 검색하고,' (길이: 29자)
... 총 10개 청크 생성됨


In [9]:

# ===================================
#  설정별 결과 비교
# ===================================
print("\n" + "="*60)
print(" 설정별 결과 요약")
print("="*60)

results = [
    ("마침표 기준 (50자)", len(chunks1), chunks1),
    ("마침표 기준 (100자)", len(chunks2), chunks2),
    ("줄바꿈 기준", len(chunks3), chunks3),
    ("공백 기준", len(chunks4), chunks4)
]

for name, count, chunks in results:
    avg_length = sum(len(chunk) for chunk in chunks) / len(chunks)
    print(f"{name:15}: {count:2}개 청크, 평균 {avg_length:.1f}자")

# ===================================
#  chunk_overlap 효과 확인
# ===================================
print("\n" + "="*60)
print(" chunk_overlap 효과 확인")
print("="*60)

# 중복 없음
splitter_no_overlap = CharacterTextSplitter(
    chunk_size=50, chunk_overlap=0, separator="."
)
chunks_no_overlap = splitter_no_overlap.split_text(text)

# 중복 있음
splitter_with_overlap = CharacterTextSplitter(
    chunk_size=50, chunk_overlap=15, separator="."
)
chunks_with_overlap = splitter_with_overlap.split_text(text)

print("\n 중복 없음 (overlap=0):")
for i, chunk in enumerate(chunks_no_overlap, 1):
    print(f"청크 {i}: '{chunk.strip()}'")

print("\n 중복 있음 (overlap=15):")
for i, chunk in enumerate(chunks_with_overlap, 1):
    print(f"청크 {i}: '{chunk.strip()}'")
    if i > 1:  # 두 번째 청크부터 중복 부분 표시
        prev_chunk = chunks_with_overlap[i-2].strip()
        curr_chunk = chunk.strip()
        # 간단한 중복 확인
        if len(prev_chunk) > 10 and len(curr_chunk) > 10:
            if prev_chunk[-10:] in curr_chunk:
                print(f"    이전 청크와 중복: '{prev_chunk[-10:]}'")

# ===================================
#  실무 팁
# ===================================
print("\n" + "="*60)
print(" 실무 활용 팁")
print("="*60)

tips = """
 청크 크기 가이드:
   • 짧은 문서: 200-500자
   • 긴 문서: 500-1000자
   • 매우 긴 문서: 1000-2000자

 chunk_overlap 가이드:
   • 일반적: 청크 크기의 10-20%
   • 맥락 중요: 청크 크기의 20-30%
   • 속도 중요: 0-10%

 separator 선택:
   • 문서: 문단(\n\n) 또는 문장(.)
   • 코드: 함수나 클래스 단위
   • 대화: 발화자 변경 지점

 RAG 최적화:
   • 너무 작으면 맥락 손실
   • 너무 크면 관련성 저하
   • 적절한 중복으로 연결성 유지
"""

print(tips)


 설정별 결과 요약
마침표 기준 (50자)   :  7개 청크, 평균 31.6자
마침표 기준 (100자)  :  5개 청크, 평균 73.2자
줄바꿈 기준         :  4개 청크, 평균 57.8자
공백 기준          : 10개 청크, 평균 25.9자

 chunk_overlap 효과 확인

 중복 없음 (overlap=0):
청크 1: 'RAG는 검색 기반의 텍스트 생성 모델입니다'
청크 2: '기존 언어 모델의 단점을 보완하고, 최신 정보를 제공합니다'
청크 3: '특히, 최신 데이터를 반영하는 데 강력한 기능을 제공합니다'
청크 4: 'RAG는 검색과 생성 단계를 포함합니다'
청크 5: '먼저 관련 문서를 검색하고, 그 다음 검색된 문서를 바탕으로 답변을 생성합니다'
청크 6: '이 방식은 환상(hallucination) 문제를 크게 줄여줍니다'
청크 7: '또한 실시간으로 최신 정보를 활용할 수 있어 매우 유용합니다'

 중복 있음 (overlap=15):
청크 1: 'RAG는 검색 기반의 텍스트 생성 모델입니다'
청크 2: '기존 언어 모델의 단점을 보완하고, 최신 정보를 제공합니다'
청크 3: '특히, 최신 데이터를 반영하는 데 강력한 기능을 제공합니다'
청크 4: 'RAG는 검색과 생성 단계를 포함합니다'
청크 5: '먼저 관련 문서를 검색하고, 그 다음 검색된 문서를 바탕으로 답변을 생성합니다'
청크 6: '이 방식은 환상(hallucination) 문제를 크게 줄여줍니다'
청크 7: '또한 실시간으로 최신 정보를 활용할 수 있어 매우 유용합니다'

 실무 활용 팁

 청크 크기 가이드:
   • 짧은 문서: 200-500자
   • 긴 문서: 500-1000자
   • 매우 긴 문서: 1000-2000자

 chunk_overlap 가이드:
   • 일반적: 청크 크기의 10-20%
   • 맥락 중요: 청크 크기의 20-30%
   • 속도 중요: 0-10%

 separator 선택:
   • 문서: 문단(

) 또는 문장(.)

### RecursiveCharacterTextSplitter
- 설정된 문자 목록을 순서대로 사용해 가장 적절한 크기로 분할

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 예제 텍스트
text = """RAG는 검색과 생성 단계를 포함하는 모델입니다.

이 모델은 검색 기반의 텍스트 생성 기능을 제공합니다.
특히, 최신 데이터를 반영하는 데 강력한 기능을 가지고 있습니다.

Transformer 모델을 기반으로 실시간 정보를 활용할 수 있으며, 기존의 단순한 생성 모델보다 더 정확한 답변을 제공합니다.

RAG의 핵심은 검색과 생성의 결합입니다! 먼저 관련 문서를 찾고, 그 정보를 바탕으로 답변을 만듭니다."""

print("원본 텍스트:")
print("-" * 50)
print(text)
print(f"\n텍스트 길이: {len(text)}자")

# ===========================================
# Recursive vs Character 비교
# ===========================================

print("\n" + "="*60)
print("RecursiveCharacterTextSplitter vs CharacterTextSplitter 비교")
print("="*60)

# 1. RecursiveCharacterTextSplitter (추천)
print("\n1. RecursiveCharacterTextSplitter (계층적 분할):")
print("-" * 45)
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=80,
    chunk_overlap=20,
    separators=["\n\n", "\n", ".", "!", "?", " ", ""]  # 우선순위 순서
)
recursive_chunks = recursive_splitter.split_text(text)

for i, chunk in enumerate(recursive_chunks):
    print(f"Chunk {i+1}: '{chunk.strip()}'")
    print(f"길이: {len(chunk)}자")
    print()

# 2. CharacterTextSplitter (비교용)
print("2. CharacterTextSplitter (단순 분할):")
print("-" * 35)
from langchain.text_splitter import CharacterTextSplitter
simple_splitter = CharacterTextSplitter(
    chunk_size=80,
    chunk_overlap=20,
    separator="."  # 하나의 구분자만 사용
)
simple_chunks = simple_splitter.split_text(text)

for i, chunk in enumerate(simple_chunks):
    print(f"Chunk {i+1}: '{chunk.strip()}'")
    print(f"길이: {len(chunk)}자")
    print()


원본 텍스트:
--------------------------------------------------
RAG는 검색과 생성 단계를 포함하는 모델입니다.

이 모델은 검색 기반의 텍스트 생성 기능을 제공합니다.
특히, 최신 데이터를 반영하는 데 강력한 기능을 가지고 있습니다.

Transformer 모델을 기반으로 실시간 정보를 활용할 수 있으며, 기존의 단순한 생성 모델보다 더 정확한 답변을 제공합니다.

RAG의 핵심은 검색과 생성의 결합입니다! 먼저 관련 문서를 찾고, 그 정보를 바탕으로 답변을 만듭니다.

텍스트 길이: 230자

RecursiveCharacterTextSplitter vs CharacterTextSplitter 비교

1. RecursiveCharacterTextSplitter (계층적 분할):
---------------------------------------------
Chunk 1: 'RAG는 검색과 생성 단계를 포함하는 모델입니다.'
길이: 27자

Chunk 2: '이 모델은 검색 기반의 텍스트 생성 기능을 제공합니다.
특히, 최신 데이터를 반영하는 데 강력한 기능을 가지고 있습니다.'
길이: 67자

Chunk 3: 'Transformer 모델을 기반으로 실시간 정보를 활용할 수 있으며, 기존의 단순한 생성 모델보다 더 정확한 답변을 제공합니다.'
길이: 72자

Chunk 4: 'RAG의 핵심은 검색과 생성의 결합입니다! 먼저 관련 문서를 찾고, 그 정보를 바탕으로 답변을 만듭니다.'
길이: 58자

2. CharacterTextSplitter (단순 분할):
-----------------------------------
Chunk 1: 'RAG는 검색과 생성 단계를 포함하는 모델입니다.

이 모델은 검색 기반의 텍스트 생성 기능을 제공합니다'
길이: 58자

Chunk 2: '특히, 최신 데이터를 반영하는 데 강력한 기능을 가지고 있습니다'
길이: 35자

Chunk 3: 'Transformer 모델을 기반으로 실

In [13]:

# ===========================================
# separators 우선순위 테스트
# ===========================================

print("="*60)
print("separators 우선순위 동작 확인")
print("="*60)

test_text = """첫 번째 문단입니다.

두 번째 문단입니다.
이 문단은 여러 문장으로 구성됩니다! 정말 흥미롭죠?

세 번째 문단입니다."""

print("테스트 텍스트:")
print(repr(test_text))  # 줄바꿈 문자까지 보이도록

# 다양한 separators 설정 테스트
separators_configs = [
    (["\n\n", "\n", ".", " "], "문단 우선"),
    (["\n", ".", " "], "줄바꿈 우선"),
    ([".", "!", "?", " "], "문장 우선"),
    ([" "], "단어 단위")
]

for separators, description in separators_configs:
    print(f"\n{description} separators={separators}:")
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=40,
        chunk_overlap=10,
        separators=separators
    )
    chunks = splitter.split_text(test_text)
    
    for i, chunk in enumerate(chunks, 1):
        print(f"  Chunk {i}: '{chunk.strip()}'")

# ===========================================
# chunk_size별 결과 비교
# ===========================================

print("\n" + "="*60)
print("chunk_size별 분할 결과 비교")
print("="*60)

chunk_sizes = [50, 100, 150]

for size in chunk_sizes:
    print(f"\nchunk_size={size}:")
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=size,
        chunk_overlap=20,
        separators=["\n\n", "\n", ".", " "]
    )
    chunks = splitter.split_text(text)
    
    print(f"총 {len(chunks)}개 청크 생성")
    avg_length = sum(len(chunk) for chunk in chunks) / len(chunks)
    print(f"평균 청크 길이: {avg_length:.1f}자")
    
    for i, chunk in enumerate(chunks, 1):
        print(f"  Chunk {i}: '{chunk.strip()[:30]}...' (길이: {len(chunk)}자)")

# ===========================================
# chunk_overlap 효과 확인
# ===========================================

print("\n" + "="*60)
print("chunk_overlap 효과 확인")
print("="*60)

overlap_values = [0, 10, 30]

for overlap in overlap_values:
    print(f"\nchunk_overlap={overlap}:")
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=80,
        chunk_overlap=overlap,
        separators=["\n\n", ".", " "]
    )
    chunks = splitter.split_text(text)
    
    print(f"총 {len(chunks)}개 청크 생성")
    for i, chunk in enumerate(chunks, 1):
        print(f"  Chunk {i}: '{chunk.strip()}'")
        
        # 중복 부분 확인
        if i > 1 and overlap > 0:
            prev_chunk = chunks[i-2].strip()
            curr_chunk = chunk.strip()
            # 간단한 중복 확인 (마지막 10자와 첫 10자 비교)
            if len(prev_chunk) >= 10 and len(curr_chunk) >= 10:
                prev_end = prev_chunk[-10:]
                curr_start = curr_chunk[:10]
                if any(word in curr_start for word in prev_end.split() if len(word) > 2):
                    print(f"    중복 감지: 이전 청크와 겹치는 부분 있음")


separators 우선순위 동작 확인
테스트 텍스트:
'첫 번째 문단입니다.\n\n두 번째 문단입니다.\n이 문단은 여러 문장으로 구성됩니다! 정말 흥미롭죠?\n\n세 번째 문단입니다.'

문단 우선 separators=['\n\n', '\n', '.', ' ']:
  Chunk 1: '첫 번째 문단입니다.'
  Chunk 2: '두 번째 문단입니다.'
  Chunk 3: '이 문단은 여러 문장으로 구성됩니다! 정말 흥미롭죠?'
  Chunk 4: '세 번째 문단입니다.'

줄바꿈 우선 separators=['\n', '.', ' ']:
  Chunk 1: '첫 번째 문단입니다.

두 번째 문단입니다.'
  Chunk 2: '이 문단은 여러 문장으로 구성됩니다! 정말 흥미롭죠?'
  Chunk 3: '세 번째 문단입니다.'

문장 우선 separators=['.', '!', '?', ' ']:
  Chunk 1: '첫 번째 문단입니다.

두 번째 문단입니다'
  Chunk 2: '.
이 문단은 여러 문장으로 구성됩니다'
  Chunk 3: '! 정말 흥미롭죠?

세 번째 문단입니다'
  Chunk 4: '.'

단어 단위 separators=[' ']:
  Chunk 1: '첫 번째 문단입니다.

두 번째 문단입니다.
이 문단은 여러 문장으로'
  Chunk 2: '여러 문장으로 구성됩니다! 정말 흥미롭죠?

세 번째 문단입니다.'

chunk_size별 분할 결과 비교

chunk_size=50:
총 9개 청크 생성
평균 청크 길이: 28.8자
  Chunk 1: 'RAG는 검색과 생성 단계를 포함하는 모델입니다....' (길이: 27자)
  Chunk 2: '이 모델은 검색 기반의 텍스트 생성 기능을 제공합니다....' (길이: 30자)
  Chunk 3: '특히, 최신 데이터를 반영하는 데 강력한 기능을 가지고...' (길이: 36자)
  Chunk 4: 'Transformer 모델을 기반으로 실시간 정보를 활...' (길이: 47자)
  Chunk 5

In [14]:

# ===========================================
# 활용 가이드
# ===========================================

print("\n" + "="*60)
print("실무 활용 가이드")
print("="*60)

print("""
RecursiveCharacterTextSplitter 사용 가이드:

1. 기본 설정 (일반적 문서):
   chunk_size=1000, chunk_overlap=200
   separators=["\n\n", "\n", ".", " "]

2. 한국어 문서 최적화:
   chunk_size=500-1000, chunk_overlap=100-200
   separators=["\n\n", "\n", ".", "。", " "]

3. 코드 문서:
   separators=["\n\n", "\n", "\t", " "]

4. 대화/채팅 로그:
   separators=["\n\n", "\n", ":", " "]

장점:
- 의미 단위로 자연스러운 분할
- 계층적 구분자로 최적화된 분할점 찾기
- 텍스트 특성에 맞는 유연한 설정

주의사항:
- chunk_size는 LLM 토큰 제한 고려
- chunk_overlap은 맥락 보존과 비용의 균형
- separators 순서가 분할 품질 결정
""")

print("\n프로그램 완료")


실무 활용 가이드

RecursiveCharacterTextSplitter 사용 가이드:

1. 기본 설정 (일반적 문서):
   chunk_size=1000, chunk_overlap=200
   separators=["

", "
", ".", " "]

2. 한국어 문서 최적화:
   chunk_size=500-1000, chunk_overlap=100-200
   separators=["

", "
", ".", "。", " "]

3. 코드 문서:
   separators=["

", "
", "	", " "]

4. 대화/채팅 로그:
   separators=["

", "
", ":", " "]

장점:
- 의미 단위로 자연스러운 분할
- 계층적 구분자로 최적화된 분할점 찾기
- 텍스트 특성에 맞는 유연한 설정

주의사항:
- chunk_size는 LLM 토큰 제한 고려
- chunk_overlap은 맥락 보존과 비용의 균형
- separators 순서가 분할 품질 결정


프로그램 완료


### TokenTextSplitter
- token기준 분할

In [16]:
# %pip install tiktoken
from langchain_text_splitters import TokenTextSplitter

# 파일 읽기
with open("./data/ai_terminology.txt", encoding="utf-8") as f:
    file = f.read()  # 파일 내용을 읽어오기

print("원본 텍스트 미리보기:\n", file[:500])  # 앞 500자 출력

# TokenTextSplitter 설정
text_splitter = TokenTextSplitter.from_tiktoken_encoder(
    chunk_size=200,  # 청크 크기
    chunk_overlap=20,  # 청크 간 겹치는 부분 추가하여 문맥 유지
    encoding_name="cl100k_base",  # OpenAI tiktoken 기본 인코딩 사용 (한글 처리 개선)
    add_start_index=True  # 각 청크의 시작 인덱스 반환
)

# 텍스트 분할 실행
texts = text_splitter.split_text(file)

# 결과 출력
print(f"\n 총 {len(texts)}개의 청크로 분할됨.")
print("\n 첫 번째 청크:\n", texts[0])

# 청크 길이 확인
for i, chunk in enumerate(texts[:5]):  # 처음 5개만 확인
    print(f"\n Chunk {i+1} (길이: {len(chunk)}):\n{chunk}")

원본 텍스트 미리보기:
 Semantic Search (의미론적 검색)

정의: 사용자의 질의를 단순한 키워드 매칭이 아니라 문맥과 의미를 분석하여 관련 정보를 반환하는 검색 방식.
예시: "우주 탐사"를 검색하면 "아폴로 11호", "화성 탐사 로버"와 같은 연관 정보가 포함된 결과를 제공함.
연관 키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

FAISS (Facebook AI Similarity Search)

정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.
연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화

Embedding (임베딩)

정의: 단어나 문장을 벡터 공간에 매핑하여 의미적으로 유사한 것들이 가까이 위치하도록 하는 기법.
예시: "강아지"와 "고양이"의 벡터 표현이 

 총 14개의 청크로 분할됨.

 첫 번째 청크:
 Semantic Search (의미론적 검색)

정의: 사용자의 질의를 단순한 키워드 매칭이 아니라 문맥과 의미를 분석하여 관련 정보를 반환하는 검색 방식.
예시: "우주 탐사"를 검색하면 "아폴로 11호", "화성 탐사 로버"와 같은 연관 정보가 포함된 결과를 제공함.
연관 키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

FAISS (Facebook AI Similarity Search)

정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대�

 Chunk 1 (길이: 270):
Semantic Search (의미론적 검색)

정의: 사용자의 질의를 단순한 키워드 매칭이 아니라 문맥과 의미를 분석하여 관련 정보를 반환하는 검색 방식.
예시: "우주 탐사"를 검색하면 "아폴로 11호", "화성 탐사 로버"와 같은 연관 정보가 포함된 결과를 제공함.
연관 키워드: 자연어

### HuggingFaceTokenizer

In [19]:
from transformers import GPT2TokenizerFast
from langchain.text_splitter import CharacterTextSplitter

# GPT-2 모델의 토크나이저 로드
hf_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# 데이터 파일 읽기
file_path = "./data/ai_terminology.txt"
with open(file_path, encoding="utf-8") as f:
    file_content = f.read()

print(" 원본 텍스트 미리보기:\n", file_content[:200])

# CharacterTextSplitter 설정 (Hugging Face 토크나이저 사용)
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    hf_tokenizer,
    chunk_size=300,  # 각 청크 크기 (토큰 기준 아님)
    chunk_overlap=50,  # 청크 간 중복 부분
)

# 텍스트 분할 수행
split_texts = text_splitter.split_text(file_content)

# 분할된 텍스트 출력
print(f"\n 총 {len(split_texts)}개의 청크로 분할됨\n")
for i, chunk in enumerate(split_texts[:5]):  # 처음 5개만 출력
    print(f" Chunk {i+1} ({len(chunk)}자):\n{chunk}\n")

# 토크나이저로 텍스트를 토큰 단위로 변환하여 확인
tokenized_example = hf_tokenizer.tokenize(split_texts[0])
print(f"\n 첫 번째 청크의 토큰 개수: {len(tokenized_example)}")
print(" 첫 번째 청크의 토큰 리스트:", tokenized_example[:20])  # 앞 20개만 출력

Created a chunk of size 321, which is longer than the specified 300
Created a chunk of size 362, which is longer than the specified 300


 원본 텍스트 미리보기:
 Semantic Search (의미론적 검색)

정의: 사용자의 질의를 단순한 키워드 매칭이 아니라 문맥과 의미를 분석하여 관련 정보를 반환하는 검색 방식.
예시: "우주 탐사"를 검색하면 "아폴로 11호", "화성 탐사 로버"와 같은 연관 정보가 포함된 결과를 제공함.
연관 키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

FAISS (Faceboo

 총 23개의 청크로 분할됨

 Chunk 1 (25자):
Semantic Search (의미론적 검색)

 Chunk 2 (157자):
정의: 사용자의 질의를 단순한 키워드 매칭이 아니라 문맥과 의미를 분석하여 관련 정보를 반환하는 검색 방식.
예시: "우주 탐사"를 검색하면 "아폴로 11호", "화성 탐사 로버"와 같은 연관 정보가 포함된 결과를 제공함.
연관 키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

 Chunk 3 (37자):
FAISS (Facebook AI Similarity Search)

 Chunk 4 (176자):
정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.
연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화

 Chunk 5 (143자):
Embedding (임베딩)

정의: 단어나 문장을 벡터 공간에 매핑하여 의미적으로 유사한 것들이 가까이 위치하도록 하는 기법.
예시: "강아지"와 "고양이"의 벡터 표현이 유사하게 위치함.
연관 키워드: 벡터화, 자연어 처리, 딥러닝

Token (토큰)


 첫 번째 청크의 토큰 개수: 23
 첫 번째 청크의 토큰 리스트: ['Sem', 'antic', 'ĠSearch', 'Ġ(', 'ìĿ', 'ĺ', 'ë', '¯', '¸', 'ë', '¡', 'ł', 'ì', 'ł', 'ģ', 'Ġ', '